# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


Aplicando la serie de Taylor a una función $f(x)$ obtenemos: 

\begin{eqnarray}
f(x) & = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,  ...(1)\\
\end{eqnarray}

Donde cada $f_{[i]}$ es un coeficiente de la serie.

Si truncamos la serie anterior se tiene:

\begin{eqnarray}
f(x) & = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k  ...(2) \\\end{eqnarray}

A partir de (2) podemos representar los coeficientes en un arreglo como el siguiente:

\begin{eqnarray}
Taylor[f_{[0]},f_{[1]},f_{[2]},...f_{[k]}]  ...(3)\end{eqnarray}

En (3) tenemos una estructura que nos recuerda a la forma del dual: 

\begin{eqnarray}
Dual(f,df) ...(4)
\end{eqnarray}

,la cual, podemos representar ahora como:

\begin{eqnarray}
Taylor[f_{[0]},f_{[1]}]  ...(5)
\end{eqnarray}

En un inicio, tuvimos que asegurar que el dual cumpliera con las siguientes propiedades:
\begin{eqnarray}
\\
Dual(c,0)  ...(a)
\\
Dual(x,1)          ...(b)
\\
\end{eqnarray}

(Donde c es una constante y en b) se utiliza la función $f(x)=x$)


Aplicando estos casos a (3):
\begin{eqnarray}
\\
Taylor[c,0,0,...,0]  ...(c)
\\
Taylor[x,1,0,...,0]  ...(d)
\\
\end{eqnarray}

Notemos que en el caso de una constante la primera entrada es justo la constante y la segunda es 0 (su primera derivada). Por otro lado, para la variable x la primera entrada es x y la segunda es 1 (su primera derivada). . 

In [2]:
"""
Definición de polinomios de Taylor, donde

cofun::Array{T,1} es el arreglo de los coeficientes de la función f.
grado::Int64 es el grado de la función
...
"""
type Taylor{T<:Number}
    # código: 
    cofun::Array{T,1} #Arreglo de los coeficientes de la función f.
    grado::Int64 #Grado de la función
    
    function Taylor(v::Vector{T},n::Int) 
        len = length(v) #Longitud del vector
        
        if len-1==n 
            return new{T}(v,n) #Si el grado ingresado es igual a los coeficientes del vector, 
                               #el grado y el vector se quedan igual.
            
        elseif len-1<n #Si el grado ingresado es mayor a los coeficientes del vector, 
                           #se completa con ceros.
            
            cv=copy(v) #Hacemos una copia del vector de entrada v, con el fin
                       #de no modificar su logitud original.  
            
            return new{T}(append!(cv,zeros(T,n-len+1)),n)
        else               #Si el grado ingresado es menor a los coeficientes del vector, 
                           #se resta un grado.
            return new{T}(v,len-1)
        end
    end
       
end

Taylor

In [3]:
methods(Taylor)

# 1 method for generic function "(::Type)":
(::Type{T}){T}(arg) at sysimg.jl:53

In [4]:
Taylor([1,2,3,4],6)

LoadError: MethodError: no method matching Taylor{T<:Number}(::Array{Int64,1}, ::Int64)[0m
Closest candidates are:
  Taylor{T<:Number}{T}(::Any) at sysimg.jl:53[0m

In [5]:
#Para que lo anterior funcione debemos especificar el tipo
Taylor{Int64}([1,2,3,4],6)

Taylor{Int64}([1,2,3,4,0,0,0],6)

In [6]:
#Con esto evitamos escribir el tipo cada vez.
Taylor{T<:Number}(v::Array{T,1},n::Int)=Taylor{T}(v,n)

Taylor{T<:Number}

In [7]:
Taylor([0,1,2],2) #caso lenght(v)=grado

Taylor{Int64}([0,1,2],2)

In [8]:
Taylor([0,1,2],3) #caso lenght(v)<grado

Taylor{Int64}([0,1,2,0],3)

In [9]:
Taylor([0,1,2],1) #caso lenght(v)>grado

Taylor{Int64}([0,1,2],2)

In [10]:
#Notemos que no fue necesario hacer un promote
Taylor([1.0,2.3,5.9,4],5)

Taylor{Float64}([1.0,2.3,5.9,4.0,0.0,0.0],5)

In [11]:
#Ahora necesitamos un método para que la suma entre objetos de tipo taylor tenga sentido.

Taylor([1,2,3,4],3)+Taylor([9,8,7,6],3) #mismo grado
Taylor([1,2,3,4],7)+Taylor([9,8],2)     #diferente grado

LoadError: MethodError: no method matching +(::Taylor{Int64}, ::Taylor{Int64})[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138[0m

Veamos cómo separar los elementos de un objeto tipo taylor

In [12]:
alfa = Taylor([0,1,2,3],5)

Taylor{Int64}([0,1,2,3,0,0],5)

In [13]:
alfa.cofun #Obtenemos solo el vector

6-element Array{Int64,1}:
 0
 1
 2
 3
 0
 0

In [14]:
alfa.cofun[4] # Obtenemos un elemento del vector

3

In [15]:
alfa.grado #Obtenemos solo el grado

5

# Operaciones

A continuación creamos los métodos necesarios para poder operar con los objetos tipo Taylor. La siguiente línea nos permite introducir las operaciones básicas de suma,resta,multiplicación,división y comparación.

In [16]:
import Base: +, -, *, /, ==,^

In [17]:
a1=Taylor([1,2,3],3);b1=Taylor([9,8,7,6],3)

Taylor{Int64}([9,8,7,6],3)

In [18]:
c1=Taylor([1,2,3,4],2);d1=Taylor([9,8],7)

Taylor{Int64}([9,8,0,0,0,0,0,0],7)

### Suma

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]}
\end{eqnarray}

In [19]:
function tsuma(a,b)
    #Comparamos la longitud de los vectores
    
    la=length(a.cofun)
    lb=length(b.cofun)
 
    if a.grado==b.grado #Si los grados son iguales
        
            grado_principal=a.grado

            suma=a.cofun+b.cofun
        
    elseif a.grado>b.grado
        
            grado_principal=a.grado #El grado pricipal es el mayor de los grados
        
            ceros=zeros(la-lb)
            b=copy(b.cofun)  
            for i=1:length(ceros)
                b=push!(b,ceros[i])
            end
            
            suma=a.cofun+b
        
    else 
       
            grado_principal=b.grado #El grado pricipal es el mayor de los grados
        
            ceros=zeros(lb-la) #Creamos un vector con ceros para igualar en tamaño
            a=copy(a.cofun)                   #al vector más chico.
           
            for i=1:length(ceros) #Agragamos los ceros al vector chico
                a=push!(a,ceros[i])
            end
            
            suma=a+b.cofun 
               
    end   
                                              
    return Taylor(suma,grado_principal)
    
end

tsuma (generic function with 1 method)

In [20]:
tsuma(a1,b1)

Taylor{Int64}([10,10,10,6],3)

In [21]:
tsuma(c1,d1)

Taylor{Int64}([10,10,3,4,0,0,0,0],7)

In [22]:
tsuma(d1,c1)

Taylor{Int64}([10,10,3,4,0,0,0,0],7)

Para tener la implementación de la suma,falta hacer que la siguiente línea tenga sentido:

In [23]:
c1+d1

LoadError: MethodError: no method matching +(::Taylor{Int64}, ::Taylor{Int64})[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138[0m

Lo cual, logramos con lo siguiente:

In [24]:
+(f::Taylor, g::Taylor) = tsuma(f,g)

+ (generic function with 165 methods)

In [25]:
c1+d1

Taylor{Int64}([10,10,3,4,0,0,0,0],7)

### Resta


\begin{eqnarray}
(f-g)_{[k]} & = & f_{[k]} - g_{[k]}
\end{eqnarray}

In [26]:
function tresta(a,b)
    #Comparamos la longitud de los vectores
    
    la=length(a.cofun)
    lb=length(b.cofun)
 
    if a.grado==b.grado #Si los grados son iguales
        
            grado_principal=a.grado

            resta=a.cofun-b.cofun
        
    elseif a.grado>b.grado
        
            grado_principal=a.grado #El grado pricipal es el mayor de los grados
        
            ceros=zeros(la-lb)
            b=copy(b.cofun)  
            for i=1:length(ceros)
                b=push!(b,ceros[i])
            end
            
            resta=a.cofun-b
        
    else 
       
            grado_principal=b.grado #El grado pricipal es el mayor de los grados
        
            ceros=zeros(lb-la) #Creamos un vector con ceros para igualar en tamaño
            a=copy(a.cofun)                   #al vector más chico.
           
            for i=1:length(ceros) #Agragamos los ceros al vector chico
                a=push!(a,ceros[i])
            end
            
            resta=a-b.cofun 
               
    end   
                                              
    return Taylor(resta,grado_principal)
    
end

tresta (generic function with 1 method)

In [27]:
tresta(a1,b1)

Taylor{Int64}([-8,-6,-4,-6],3)

In [28]:
tresta(c1,b1)

Taylor{Int64}([-8,-6,-4,-2],3)

In [29]:
tresta(b1,c1)

Taylor{Int64}([8,6,4,2],3)

Resta de objetos tipo Taylor:

In [30]:
-(f::Taylor, g::Taylor) = tresta(f,g)

- (generic function with 192 methods)

In [31]:
b1-c1

Taylor{Int64}([8,6,4,2],3)

In [32]:
Taylor([1,2,3],3)-Taylor([1,2,3],3)

Taylor{Int64}([0,0,0,0],3)

### Multiplicación

\begin{eqnarray}
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]}
\end{eqnarray}

Debemos notar que, dado un vector $v$, la entrada $v[0]$ no está definida, por lo que la suma no podrá iniciar en cero. Por ello cambiaremos los índices de la expresión anterior de modo tal que la suma inicie con $i=1$ 

\begin{eqnarray}
(f \cdot g)_{[k]} & = & \sum_{i=1}^k f_{[i]} \,g_{[k+1-i]}
\end{eqnarray}

In [33]:
function tmulti(a,b)
    
#=---------------------Multiplicación de los vectores-------------------------=#
        
        grado_principal=a.grado+b.grado #obtenemos el grado  principal
    
        #Copiamos los vectores para no modificar los originales
        a=copy(a.cofun)
        b=copy(b.cofun)
        
        #Para los polinomios p = [1,1] y q = [1-1]
        #Si creamos las entradas extra a[3]=b[3]=0 , de la siguiente forma:
                     #a=push!(a,0)
                     #b=push!(b,0)
        #De este modo, el método nos da todas las entradas del polinomio resultante.
        #Sin embargo, esto no funciona para otros polinomios.
       
        
        #Obtenemos la longitud de los vectores
        la=length(a)
        lb=length(b)
    
        suma=zeros(la) #Aquí estamos truncando la multiplicación, 
                       #pues el arreglo del resultado
                       #puede ser mayor que el de los vectores de entrada. 
#************************************************************************************* 
    if la==lb
        
            for k=1:la
                for i=1:k
                    suma[k]+=a[i]*b[k+1-i]
                end
            end 
        
    elseif la<lb  
                #-------------Ajuste de logitud--------------

            lc=lb-la #Calculamos la diferencia de las longitudes
            ceros=zeros(lc) #Creamos un vector con ceros para igualar en tamaño
                            #al vector más chico.

            for i=1:length(ceros) #Agregamos los ceros al vector chico
                a=push!(a,ceros[i])
            end
                #---------------------------------------------
            for k=1:la
                for i=1:k
                    suma[k]+=a[i]*b[k+1-i]
                end
            end 
        
     else
             #-------------Ajuste de logitud---------------
            lc=la-lb
            ceros=zeros(lc)       

            for i=1:length(ceros)
                b=push!(b,ceros[i])
            end
             #---------------------------------------------  
            for k=1:la
                for i=1:k
                    suma[k]+=a[i]*b[k+1-i]
                end
            end 
        
    end  
        
    return Taylor(suma,grado_principal)
    
end   

tmulti (generic function with 1 method)

In [34]:
e1=Taylor([1,1],1);e2=Taylor([1,-1],1)

Taylor{Int64}([1,-1],1)

In [35]:
tmulti(e1,e2)

Taylor{Float64}([1.0,0.0,0.0],2)

In [36]:
d1=Taylor([1,0,0,-1],3)

Taylor{Int64}([1,0,0,-1],3)

In [37]:
tmulti(d1,d1)

Taylor{Float64}([1.0,0.0,0.0,-2.0,0.0,0.0,0.0],6)

In [38]:
tmulti(e1,d1)

Taylor{Float64}([1.0,1.0,0.0,0.0,0.0],4)

In [39]:
tmulti(d1,e1)

Taylor{Float64}([1.0,1.0,0.0,-1.0,0.0],4)

In [40]:
e2=Taylor([1,1],1)

Taylor{Int64}([1,1],1)

In [41]:
tmulti(e2,e2)

Taylor{Float64}([1.0,2.0,0.0],2)

Multiplicación de objetos tipo Taylor:

In [42]:
*(f::Taylor, g::Taylor) = tmulti(f,g)

* (generic function with 151 methods)

In [43]:
Taylor([1,1,1,1],1)*Taylor([8,8,8,8],3)

Taylor{Float64}([8.0,16.0,24.0,32.0,0.0,0.0,0.0],6)

In [44]:
Taylor([8,8,8,8],3)*Taylor([1,1,1,1],1)

Taylor{Float64}([8.0,16.0,24.0,32.0,0.0,0.0,0.0],6)

In [45]:
Taylor([8,8,8,8],4)*Taylor([1,1,1,1],3)

Taylor{Float64}([8.0,16.0,24.0,32.0,24.0,0.0,0.0,0.0],7)

### División


\begin{eqnarray}
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) 
\end{eqnarray}

Al igual que en el caso de la multiplicación,la suma no podrá iniciar en cero. Por ello cambiaremos los índices de la expresión anterior de modo tal que la suma inicie con $i=1$ 

\begin{eqnarray}
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[1]}}
\Big( f_{[k]} - \sum_{i=1}^{k} \big(\frac{f}{g}\big)_{[i]} \, g_{[k+1-i]} \Big) 
\end{eqnarray}

In [46]:
function tdiv(a,b)
  #=---------------------División de los vectores-------------------------=#  
       
        grado_principal=a.grado-b.grado #obtenemos el grado  principal
    
if grado_principal<0
        return println("grado negativo")
end


        #Copiamos los vectores para no modificar los originales
        f=copy(a.cofun)
        g=copy(b.cofun)
        
        #Obtenemos la longitud de los vectores
        lenf=length(f)
        leng=length(g)
   #=------------------Ajuste de la longitud------------------------------=#     
    if lenf<leng

            lenc=leng-lenf #Calculamos la diferencia de las longitudes
            ceros=zeros(lenc) #Creamos un vector con ceros para igualar en tamaño
                            #al vector más chico.

            for i=1:length(ceros) #Agregamos los ceros al vector chico
                f=push!(f,ceros[i])
            end
    
    elseif lenf<leng

            lenc=lenf-leng
            ceros=zeros(lenc)       

            for i=1:length(ceros)
                g=push!(g,ceros[i])
            end
            
    end
 #************************************************************************************* 
divi = zeros(lenf)  

    for k=1:length(divi) #(era -1)
        suma=0 #Valor inicial de la suma
        for i=1:k-1
            suma+=(divi[i])*g[k+1-i]
        end
        divi[k]=(1/g[1])*(f[k]-suma)
    end
    for i=1:grado_principal+2:length(lenf)
        if divi[i]==0
            deleteat!(cociente,i)
    end
        end    
    return Taylor(divi,grado_principal)
end

tdiv (generic function with 1 method)

In [47]:
m1=Taylor([1,1,1,1],3);m2=Taylor([1,1],1)

Taylor{Int64}([1,1],1)

In [48]:
m4=Taylor([1,2,3,4,5],4)

Taylor{Int64}([1,2,3,4,5],4)

In [49]:
tdiv(m4,m1)

LoadError: BoundsError: attempt to access 4-element Array{Int64,1} at index [5]

División de objetos tipo Taylor

In [50]:
/(f::Taylor, g::Taylor) = tdiv(f,g)

/ (generic function with 55 methods)

In [51]:
Taylor([1,2,3,4],3)/Taylor([4,3,2,1],3)

Taylor{Float64}([0.25,0.3125,0.390625,0.488281],3)

In [52]:
Taylor([1,2,3,1],3)/Taylor([1,2,2,1],7)

grado negativo


In [53]:
Taylor([1,1,1,1],3)/Taylor([1,1,1,1],3)

Taylor{Float64}([1.0,0.0,0.0,0.0],3)

In [54]:
a1

Taylor{Int64}([1,2,3,0],3)

In [55]:
b1

Taylor{Int64}([9,8,7,6],3)

In [56]:
a1/b1

Taylor{Float64}([0.111111,0.123457,0.137174,-0.292029],3)

In [57]:
b1

Taylor{Int64}([9,8,7,6],3)

In [58]:
d1

Taylor{Int64}([1,0,0,-1],3)

In [59]:
b1/d1

Taylor{Float64}([9.0,8.0,7.0,15.0],3)

In [60]:
uno = Taylor([1], 4)

Taylor{Int64}([1,0,0,0,0],4)

In [61]:
x = Taylor([0,1], 4)

Taylor{Int64}([0,1,0,0,0],4)

In [62]:
uno / (uno - x)

Taylor{Float64}([1.0,1.0,1.0,1.0,1.0],4)

### Comparación

In [63]:
function tcomp(a,b)
    if a.cofun==b.cofun
        return true
    else
        return false
    end    
end    

tcomp (generic function with 1 method)

In [64]:
a1

Taylor{Int64}([1,2,3,0],3)

In [65]:
a11 = Taylor([1.0,2.0,3.0,0.0],3)

Taylor{Float64}([1.0,2.0,3.0,0.0],3)

In [66]:
d1

Taylor{Int64}([1,0,0,-1],3)

In [67]:
tcomp(a1,a1)

true

In [68]:
tcomp(a1,d1)

false

In [69]:
tcomp(a1,a11)

true

In [70]:
==(f::Taylor, g::Taylor) = tcomp(f,g)

== (generic function with 120 methods)

In [71]:
a1==a1

true

In [72]:
a = Taylor([0,1], 3)

Taylor{Int64}([0,1,0,0],3)

In [73]:
b = Taylor([0,1], 3)

Taylor{Int64}([0,1,0,0],3)

In [74]:
a==b

true

In [75]:
#=import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función
+(f::Taylor, g::Taylor) = Taylor(f.cofun+g.cofun)
-(f::Taylor, g::Taylor) = Taylor(f.cofun-g.cofun)=#

In [1]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("taylor.jl")

using Ty

@testset "pruebas de operaciones para Taylor" begin
    
    @test Taylor([1,2,3],3)+Taylor([9,8,7,6],3)==Taylor([10,10,10,6],3)
    @test Taylor([10,10,10,6],3)+Taylor([1,2,3,4,5],6)==Taylor([11,12,13,10,5,0,0],6)
    @test Taylor([1,2,3,4,5],6)+Taylor([1.0,2.0,3.0,4.0,5.0],4)==Taylor([2.0,4.0,6.0,8.0,10.0,0.0,0.0],6)
    @test Taylor([1,2,3,4,5],6)+Taylor([-1,-2,-3,-4,5],6)==Taylor([0,0,0,0,10,0,0],6)
    @test Taylor([1,2,3],3)-Taylor([9,8,7,6],3)==Taylor([-8,-6,-4,-6],3)
    @test Taylor([10,10,10,6],3)-Taylor([1,2,3,4,5],6)==Taylor([9,8,7,2,-5,0,0],6)
    @test Taylor([1,2,3,4,5],6)-Taylor([1.0,2.0,3.0,4.0,5.0],4)==Taylor([0.0,0.0,0.0,0.0,0.0,0.0,0.0],6)
    @test Taylor([1,2,3,4,5],6)-Taylor([-1,-2,-3,-4,5],6)==Taylor([2,4,6,8,0,0,0],6)
    @test Taylor([1,1,1,1],1)*Taylor([8,8,8,8],3)==Taylor([8.0,16.0,24.0,32.0,0.0,0.0,0.0],6)
    @test Taylor([8,8,8,8],4)*Taylor([1,1,1,1],3)==Taylor([8.0,16.0,24.0,32.0,24.0,0.0,0.0,0.0],7)
    @test Taylor([8,7,6,5],4)*Taylor([1.0,2.0,3.0,1],3)==Taylor([8.0,23.0,44.0,46.0,35.0,0.0,0.0,0.0],7)
    @test Taylor([8,7,-6,-5],4)*Taylor([1.0,-2.0,3.0,-1],3)==Taylor([8.0,-9.0,4.0,20.0,-15.0,0.0,0.0,0.0],7)
    @test Taylor([1,1,3,4],3)/Taylor([1,3,2,1],3)==Taylor([1.0,-2.0,7.0,-14.0],3)
    @test Taylor([1,1,1,1],3)/Taylor([1,1,1,1],3)==Taylor([1.0,0.0,0.0,0.0],3)
    @test Taylor([1,0,3,0],3)/Taylor([1,0,0,6],3)==Taylor{Float64}([1.0,0.0,3.0,-6.0],3)
    @test Taylor([9,8,7,6],3)/Taylor{Int64}([1,0,0,-1],3)==Taylor([9.0,8.0,7.0,15.0],3)
    @test Taylor([1,1],2)^3==Taylor{Int64}([1,3,3,1,0,0,0],6)
    @test Taylor([1,2,3,4],3)^3==Taylor([1,6,21,56,111,0,0,0,0,0],9)
    @test Taylor([1,2],1)^8==Taylor{Int64}([1,16,112,0,0,0,0,0,0],8)
end



Test Summary:                      | Pass  Total
  pruebas de operaciones para Taylor |   19     19


Base.Test.DefaultTestSet("pruebas de operaciones para Taylor",Any[],19,false)

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.

In [76]:
function elevar(pol,expo)
    grado_principal = expo*pol.grado #El grado principal es el producto del grado y el exponente.
    poly=copy(pol.cofun) #Hacemos una copia del vector del polinomio
    
    lp=length(grado_principal+1)
   # ceros=zeros(lp+(grado_principal-4))       
    ceros=zeros(lp) #Creamos un arreglo de ceros cuya longitud es determinada por el grado_principal 
    for i=1:length(ceros)
        poly=push!(poly,ceros[i]) #Agregamos ceros al arreglo para que tenga la longitud 
                                  #ideal para el resultado
    end
        
    p=zeros(poly) #Creamos un arreglo de ceros de la longitud del vector anterior

    p[1]=(poly[1])^expo #Primer valor del vector resultado.
    
    for k=1:length(poly)-1
        suma=0 #Valor inicial de la suma
        for i in 1:k-1
            suma+=(expo*k-(expo+1)*i)*poly[k+1-i]*p[i+1]
        end
        
        p[k+1]=p[1]*expo*(poly[k+1]/poly[1])+(1/(k*poly[1]))*suma
    end
    
    return Taylor(p,grado_principal)
end

elevar (generic function with 1 method)

In [77]:
h1=Taylor([1,2,3,4],3)

Taylor{Int64}([1,2,3,4],3)

In [78]:
elevar(h1,3) #El resultado completo se obtiene con ceros=zeros(lp+(grado_principal-4))

Taylor{Int64}([1,6,21,56,111,0,0,0,0,0],9)

In [79]:
elevar(h1,3) #El resultado completo se obtiene con ceros=zeros(lp+(grado_principal-4))

Taylor{Int64}([1,6,21,56,111,0,0,0,0,0],9)

In [80]:
h2=Taylor([1,2],1)

Taylor{Int64}([1,2],1)

In [81]:
elevar(h2,8)

Taylor{Int64}([1,16,112,0,0,0,0,0,0],8)

In [82]:
a = Taylor([1,1],2)

Taylor{Int64}([1,1,0],2)

In [83]:
a^3

Taylor{Float64}([1.0,3.0,3.0,0.0,0.0,0.0,0.0],6)

In [84]:
elevar(a,3)

Taylor{Int64}([1,3,3,1,0,0,0],6)

In [85]:
^(f::Taylor, exp::Int) = elevar(f,exp)

^ (generic function with 49 methods)

In [86]:
a^3

Taylor{Int64}([1,3,3,1,0,0,0],6)